
# Estudiantes:
## Cristian Armando Larios Bravo

# Red Neuronal: **RNC**
# Dataset: **Papaya Ringspot**


In [11]:
# Importacion de librerias
%matplotlib inline

# import matplotlib.pyplot as plt
import os
import torch
# from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.nn import functional as Fun
import torch.nn as nn

In [12]:
class RNC(nn.Module):
    def __init__(self, num_classes):
        super(RNC, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=224, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, stride=1, padding=1)
        self.drop = nn.Dropout2d(p=0.2)

        # Capa totalmente conectada (se actualizará dinámicamente)
        self.fc = nn.Linear(in_features=224 * 28 * 28, out_features=num_classes)

    def forward(self, x):
        x = Fun.relu(self.conv1(x))
        x = self.pool(x)
        x = Fun.relu(self.conv2(x))
        x = self.pool(x)
        x = Fun.relu(self.conv3(x))
        x = self.pool(x)
        
        x = self.drop(x)
        # print("Shape after convolutions and pooling:", x.shape)
        print("Forma después de convoluciones y agrupamiento:", x.shape)
        
        x = torch.flatten(x, start_dim=1)
        # print("Shape after flattening:", x.shape)
        print("Forma después de aplanar:", x.shape)
        
        x = self.fc(x)
        return Fun.log_softmax(x, dim=1)


In [13]:
def train(model: RNC, device, train_loader, optimizer, epoch):
  # Set model to training mode
  model.train()
  train_loss = 0
  print("Época:", epoch)

  # Process the image in batches
  for batch_idx, (data, target) in enumerate(train_loader):
    # Move the data to the selected device
    data, target = data.to(device), target.to(device)

    # Reset the optimizer
    optimizer.zero_grad()

    # Push the data forward through the model layers
    output = model(data)

    # Get the loss
    loss = Fun.nll_loss(output, target)

    # Keep a running total
    train_loss += loss.item()

    # Backpropagate
    loss.backward()
    optimizer.step()
    # train_loss += Fun.nll_loss(output, target, size_average=False).data.item()

    # return average loss for epoch
    avg_loss = train_loss / (batch_idx+1)
  # print('\nTrain set: Average loss: {:.6f}'.format(avg_loss))
  print("\nConjunto de entrenamiento: Pérdida promedio: {:.6f}".format(avg_loss))
  return avg_loss

In [14]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data transformations (if needed)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # Resize if necessary
    transforms.ToTensor(),
])

# Create datasets using ImageFolder
train_dataset = datasets.ImageFolder(root='Cards/Train', transform=data_transforms)
test_dataset = datasets.ImageFolder(root='Cards/Test', transform=data_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
modelo = RNC(num_classes=2)
optimizer = optim.Adam(modelo.parameters(), lr=0.001)
avg_loss = train(modelo, 'cpu', train_loader, optimizer, epoch=100)

Época: 1
Forma después de convoluciones y agrupamiento: torch.Size([32, 224, 28, 28])
Forma después de aplanar: torch.Size([32, 175616])
Forma después de convoluciones y agrupamiento: torch.Size([32, 224, 28, 28])
Forma después de aplanar: torch.Size([32, 175616])
Forma después de convoluciones y agrupamiento: torch.Size([32, 224, 28, 28])
Forma después de aplanar: torch.Size([32, 175616])
Forma después de convoluciones y agrupamiento: torch.Size([32, 224, 28, 28])
Forma después de aplanar: torch.Size([32, 175616])
Forma después de convoluciones y agrupamiento: torch.Size([32, 224, 28, 28])
Forma después de aplanar: torch.Size([32, 175616])
Forma después de convoluciones y agrupamiento: torch.Size([16, 224, 28, 28])
Forma después de aplanar: torch.Size([16, 175616])

Conjunto de entrenamiento: Pérdida promedio: 0.111401


In [16]:
def test(model: RNC, device, test_loader):
    model.eval()  # Configura el modelo en modo de evaluación
    test_loss = 0
    correct = 0
    with torch.no_grad():  # No necesitamos calcular gradientes
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += Fun.nll_loss(output, target, reduction='sum').item()  # Suma el loss por lote
            pred = output.argmax(dim=1, keepdim=True)  # Obtén la clase predicha
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Calcula el promedio de pérdida y precisión
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    print('\nConjunto de prueba: Pérdida promedio: {:.4f}, Precisión: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return test_loss, accuracy

In [17]:
device = 'cpu'  # O 'cuda' si tienes una GPU disponible
test_loss, test_accuracy = test(modelo, device, test_loader)

Forma después de convoluciones y agrupamiento: torch.Size([5, 224, 28, 28])
Forma después de aplanar: torch.Size([5, 175616])

Conjunto de prueba: Pérdida promedio: 0.0000, Precisión: 5/5 (100%)



In [18]:
torch.save(modelo.state_dict(), "modelo_rnc.pth")
print("Modelo guardado en 'modelo_rnc.pth'")

Modelo guardado en 'modelo_rnc.pth'


In [19]:
modelo_cargado = RNC(num_classes=2)  # Asegúrate de definir correctamente la arquitectura
modelo_cargado.load_state_dict(torch.load("modelo_rnc.pth"))
modelo_cargado.eval()
print("Modelo cargado correctamente")

Modelo cargado correctamente


C:\Users\Cristian\AppData\Local\Temp\ipykernel_45680\1251869089.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo_cargado.load_state_dict(torch.load("modelo_rnc.pth

In [20]:
test_loss2, test_accuracy2 = test(modelo_cargado, device, test_loader)

Forma después de convoluciones y agrupamiento: torch.Size([5, 224, 28, 28])
Forma después de aplanar: torch.Size([5, 175616])

Conjunto de prueba: Pérdida promedio: 0.0000, Precisión: 5/5 (100%)

